# Классификация текстов: спам-фильтр для SMS
В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

## Задание

1. Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).


2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".


4. Используя **sklearn.feature_extraction.text.CountVectorizer** со стандартными настройками, получите из списка текстов матрицу признаков X.


5. Оцените качество классификации текстов с помощью **LogisticRegression()** с параметрами по умолчанию, используя **sklearn.model_selection.cross_val_score** и посчитав среднее арифметическое качества на отдельных fold'ах. Установите **random_state=2**. Параметр **cv** задайте равным **10**. В качестве метрики качества используйте **f1-меру**. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.


6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:  

    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

    "FreeMsg: Txt: claim your reward of 3 hours talk time"

    "Have you visited the last lecture on physics?"

    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

    "Only 99$"

    Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.


7. Задайте в **CountVectorizer** параметр **ngram_range=(2,2)**, затем **ngram_range=(3,3)**, затем **ngram_range=(1,3)**. Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.


8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии **MultinomialNB()**. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

    По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.


9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из **TfidfVectorizer** на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [5]:
with open('datasets/SMSSpamCollection.txt') as f:
    messages = f.readlines()
print(messages[:10])

['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'ham\tOk lar... Joking wif u oni...\n', "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'ham\tU dun say so early hor... U c already then say...\n', "ham\tNah I don't think he goes to usf, he lives around here though\n", "spam\tFreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv\n", 'ham\tEven my brother is not like to speak with me. They treat me like aids patent.\n', "ham\tAs per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune\n", 'spam\tWINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 0906170

Подготавливаем данные:

In [15]:
labels, texts = [], []
for message in messages:
    label, text = message.split(maxsplit=1) 
    labels.append(label)
    texts.append(text.strip())
    
labels = [1 if label == 'spam' else 0 for label in labels]

In [17]:
bow_transformer = CountVectorizer()
X = bow_transformer.fit_transform(texts)

Оцениваем качество с помощью логистической регрессии:

In [53]:
lr_classifier = LogisticRegression(random_state=2)
q5 = cross_val_score(lr_classifier, X, labels, cv=10, scoring='f1').mean()
q5

0.9251382558648837

In [55]:
with open('q5.txt', 'w') as f:
    f.write(str(round(q5, 1)))

Обучим логистическую регрессию и спрогнозируем на некоторых сообщениях классы:

In [28]:
test_texts = [
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]
X_test = bow_transformer.transform(test_texts)

In [31]:
lr_classifier.fit(X, labels)
predictions = lr_classifier.predict(X_test)
predictions

array([1, 1, 0, 0, 0])

In [32]:
with open('q6.txt', 'w') as f:
    f.write(' '.join([str(label) for label in predictions]))

Оценим качество лог. регрессии на разных n-граммах:

In [41]:
scores_lr = []
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    bow_transformer = CountVectorizer(ngram_range=ngram_range)
    X = bow_transformer.fit_transform(texts)
    f1_score = round(cross_val_score(lr_classifier, X, labels, scoring='f1', cv=10).mean(), 2)
    scores_lr.append(f1_score)
    print(f'ngram_range = {ngram_range}, f1 score = {f1_score}')

ngram_range = (2, 2), f1 score = 0.82
ngram_range = (3, 3), f1 score = 0.73
ngram_range = (1, 3), f1 score = 0.93


In [42]:
with open('q7.txt', 'w') as f:
    f.write(' '.join([str(label) for label in scores_lr]))

Аналогично оценим качество наивного Байеса на разных n-граммах:

In [43]:
scores_bayes = []
bayes_classifier = MultinomialNB()
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    bow_transformer = CountVectorizer(ngram_range=ngram_range)
    X = bow_transformer.fit_transform(texts)
    f1_score = round(cross_val_score(bayes_classifier, X, labels, scoring='f1', cv=10).mean(), 2)
    scores_bayes.append(f1_score)
    print(f'ngram_range = {ngram_range}, f1 score = {f1_score}')

ngram_range = (2, 2), f1 score = 0.65
ngram_range = (3, 3), f1 score = 0.38
ngram_range = (1, 3), f1 score = 0.89


In [44]:
with open('q8.txt', 'w') as f:
    f.write(' '.join([str(label) for label in scores_bayes]))

Попробуем TfidfVectorizer вместо CountVectorizer: 

In [47]:
X_tfidf = TfidfVectorizer().fit_transform(texts)

In [56]:
q9 = cross_val_score(lr_classifier, X_tfidf, labels, cv=10, scoring='f1').mean()
print('Count vectorizer:', q5)
print('Tf-idf vectorizer:', q9)

Count vectorizer: 0.9251382558648837
Tf-idf vectorizer: 0.8528599554172456


In [57]:
with open('q9.txt', 'w') as f:
    f.write('-1')